In [ ]:
!pip install chromadb

In [13]:
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')

# IMPORTANDO AS BIBLIOTECAS NECESSÁRIAS

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings

# DIVISÃO DO LIVRO EM CHUNKS

In [2]:
path = "/content/Livro-Cardiologia.pdf"

loader = PyPDFLoader(path)
pages = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(pages)

# CARREGANDO O MODELO DE EMBEDDINGS

In [4]:
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
CHROMA_PATH = "ragV1-lm"
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

In [8]:
query = 'O que é o infarto agudo do miocárdio?'

In [9]:
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

# PROMPT

In [10]:
PROMPT_TEMPLATE = """
## 1. PAPEL E OBJETIVO
Você é um assistente de IA especializado em analisar e resumir informações sobre cardiologia. Seu único objetivo é responder às perguntas do usuário baseando-se estritamente no texto fornecido no {context}. Você NÃO é um médico e não deve, em hipótese alguma, fornecer conselhos, diagnósticos ou sugerir tratamentos.

## 2. CONTEXTO DA INFORMAÇÃO
{context}

## 3. PERGUNTA DO USUÁRIO
{question}

## 4. DIRETRIZES PARA A RESPOSTA
- **Fidelidade ao Texto:** Sua resposta deve ser 100% baseada nas informações contidas no {context}. Não infira, não adicione conhecimento externo e não complete informações que não estejam presentes.
- **Clareza e Detalhe:** Forneça uma resposta detalhada e bem explicada. Se o contexto mencionar termos técnicos (ex: "inversão da onda T", "complexo QRS largo"), explique o que eles significam de forma simples, conforme descrito no próprio texto.
- **Estrutura:** Organize a resposta de forma clara. Use parágrafos curtos e, se for útil, utilize listas com marcadores (bullet points) para apresentar características, sintomas ou passos.
- **Linguagem Natural:** Integre a informação do contexto na sua resposta de forma fluida. Não use frases como "segundo o contexto" ou "o texto menciona que".

## 5. AVISO OBRIGATÓRIO
Ao final de CADA resposta, inclua a seguinte isenção de responsabilidade, exatamente como está escrita abaixo e em uma nova linha:

---
**AVISO:** Esta informação é gerada por IA para fins educacionais e não substitui uma consulta médica. Para diagnóstico e tratamento, consulte sempre um cardiologista ou outro profissional de saúde qualificado.
"""

# RESPOSTA DO MODELO

In [15]:
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

model = ChatOpenAI(api_key=openai_api_key, model="gpt-4o")
response = model.invoke(prompt)
response_text = response.content

In [16]:
print(response_text)

O infarto agudo do miocárdio (IAM) é caracterizado pela morte das células do miocárdio devido à isquemia, ou seja, uma insuficiência de oxigênio no músculo cardíaco. Durante a fase aguda do infarto, os miócitos, que são as células do músculo cardíaco, morrem na área afetada. Isso leva a uma resposta inflamatória, com remoção dos tecidos mortos e subsequente reparo, resultando na formação de uma cicatriz.

O diagnóstico clínico de IAM exige a presença de uma síndrome clínica que indique isquemia do miocárdio, acompanhada por evidências de necrose miocárdica. Essa evidência pode ser obtida por meio de exames bioquímicos, como a detecção de biomarcadores cardíacos, eletrocardiogramas, ou por técnicas de imagem. A sensibilidade e especificidade dessas ferramentas de diagnóstico podem variar.

O infarto do miocárdio geralmente ocorre quando a demanda de oxigênio do miocárdio excede a oferta, frequentemente devido à aterosclerose coronariana ou espasmos coronarianos. Causas menos comuns incl